In [1]:
model_file = 'testsdss'
import joblib
model = joblib.load(model_file+'.P')
x = joblib.load(model_file+'Params.P')
binsize, classes, colours = x['binsize'], x['classes'], x['colours']
print binsize, classes , colours
from sklearn.neural_network import MLPClassifier as mlp
import sys
sys.path.append("/home/watkinsz/Desktop/For_Zack/ML")
sys.path.append("/home/watkinsz/Desktop/For_Zack/ML/FeH")
from fitsParser.fitsParser import fitsParser
import pandas as pd
import numpy as np
import re
from numpy import linspace
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from modelValidator import psMetric, cut, shuffleDwarfs, plotRes, plot, norm_inputs, monte_carlo, printTable, transformer_man
from sklearn.gaussian_process import GaussianProcessRegressor as gpr
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from function import save_data
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Nadam

def getMags(colour):
    return colour[:colour.index('-')], colour[colour.index('-')+1:]

sdss = fitsParser(includefile = 'FeHdata.txt')
cut(sdss, metThresh = -5)
sdss.makeColours(N=10)
colours = sdss.colours
conversion = {
     'u_0' : 'u',
    'CaHK_0' : 'CaHK' 
}
conversion.update({x+'_fpsf' : x for x in 'grizy'})
for col in sdss.colours:
    mag1, mag2 = getMags(col)
    sdss.data[conversion[mag1]+'-'+conversion[mag2]] = sdss.data[col]

for field in sdss.data.keys():
    if field in conversion:
        sdss.data[conversion[field]]= sdss.data[field]
sdss.data = sdss.data.sample(frac=1)


sdss.data['test_flag'] = 0

classes = [-2 - binsize*i for i in range(int((3-2)/binsize)+1)]     

Using TensorFlow backend.


0.5 [-2.0, -2.5, -3.0] ['u_0-CaHK_0', 'u_0-g_fpsf', 'u_0-r_fpsf', 'u_0-i_fpsf', 'u_0-z_fpsf', 'u_0-y_fpsf', 'CaHK_0-g_fpsf', 'CaHK_0-r_fpsf', 'CaHK_0-i_fpsf', 'CaHK_0-z_fpsf', 'CaHK_0-y_fpsf', 'g_fpsf-r_fpsf', 'g_fpsf-i_fpsf', 'g_fpsf-z_fpsf', 'g_fpsf-y_fpsf', 'r_fpsf-i_fpsf', 'r_fpsf-z_fpsf', 'r_fpsf-y_fpsf', 'i_fpsf-z_fpsf', 'i_fpsf-y_fpsf', 'z_fpsf-y_fpsf']

Data saved

condition on dg_fpsf cut 22 inputs
condition on dr_fpsf cut 23 inputs
condition on di_fpsf cut 6 inputs
condition on dz_fpsf cut 2 inputs
condition on dy_fpsf cut 14 inputs
condition on dCaHK_0 cut 14 inputs
condition on du_0 cut 161 inputs
condition on dFeHadop cut 301 inputs
11.45% of inputs cut


In [8]:
transformer = transformer_man([-2,-2.5,-3],
    lambda x, c :  x == c, 
    lambda x : -2-binsize*x.index(1))

putbin = lambda data : [x + min([c-x for c in classes if c>x]) if x<-2 else -2 for x in data]
pretrans = lambda data : [[1 if False not in [x[i] > y or y == x[i] for y in x ] else 0 for i in range(len(x))] for x in data] 
pretrans2 = lambda data : [[0,0,1] if x[2]>0.6 else [0,1,0] if x[1]>0.33 else [1,0,0] for x in data]
z = model.predict(sdss.data[colours])
sdss.data['FeH_pred'] = transformer.inverse_transform(pretrans(z))
sdss.data['FeHround']= putbin(sdss.data['FeHadop'])


sdss.data['FeHadop'] = sdss.data['FeHround']
i = list(classes)

def conf_matrix(data, classes, adop = 'FeHround', pred = 'FeH_pred'):
    return [[len(data[(data[adop]==c1)&(data[pred]==c2)])*1.0/len(data[(data[adop]==c1)])
             if len(data[(data[adop]==c1)])!= 0 else -1
             for c2 in classes] for c1 in classes]
data = sdss.data
pur, com = psMetric(data, inc = i)
print printTable(i ,pur,com)
print pd.DataFrame(conf_matrix(data, classes), columns = classes, index = classes) , '\n'      
             
      


,FeHround,FeHadop
1757,-2.5,-3.0
32698,-2.5,-3.0
5455,-2.5,-3.0
37950,-2.5,-3.0
34793,-2.5,-3.0
32930,-2.5,-3.0
39485,-2.5,-3.0
31194,-2.5,-3.0
38425,-2.5,-3.0
2270,-2.5,-3.0


| [Fe/H]< | purity | completness |
----------------------------------
| -2.0    | 1.0000 | 1.0000      |
| -2.5    | 0.0197 | 0.5942      |
| -3.0    | 0.0000 | -1.0000     |

          -2.0      -2.5      -3.0
-2.0  0.942528  0.037696  0.019776
-2.5  0.405797  0.347826  0.246377
-3.0 -1.000000 -1.000000 -1.000000 

